In [1]:
import OpenOPC as OP
import threading as th
import pywintypes
import pyodbc
import time
pywintypes.datetime=pywintypes.TimeType

### Definicion de variables globales

In [2]:
opcserver = "ABB.AC800MC_OpcDaSarver.3"
#mainTagname = "Applications.CAPEM"
#lista de comandos:
#cerrar: finaliza el programa
close="5"
#iniciar: inicia la medicion
start= "1"
#detener: detiene la medicion
stop="2"
#Diccionario para mensajes luego de los comandos
comdic={start:"Interfaz iniciada", stop:"Interfaz detenida", close:"Cerrando programa"}
comando=stop
comprev=comando
variables=[]

Conecto con el servidor OPC

In [3]:
opc=OP.client()
opc.connect('ABB.AC800MC_OpcDaServer.3')

Conecto con base de datos

In [4]:
def conectardb():
    global cursor
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=DESKTOP-820175M\SQLEXPRESS;'
                          'Database=capem;'
                          'Trusted_Connection=yes;')

    cursor=conn.cursor()

Estos comandos se ejecutan para crear la tabla y para crear por primera vez el dato

In [5]:
#comando("create table capem.dbo.datos(id int, b1 bit not null, int1 int not null)")
#comando("insert into capem.dbo.datos values (1,0,0)")

### Funciones


In [6]:
## Funcion que permite ingresar comandos desde consola
def comandos():
    "Funcion que permite ingresar comandos desde consola"
    global comando, close, comdic, comprev
    while comando!=close:
        comando1 =input("\ningrese comando:\n  1. Iniciar interfaz\n  2. Detener interfaz\n  5. Salir\n")
        if comando1 in comdic:
            comprev=comando
            comando=comando1
            print(comdic[comando])
        else:
            print("el comando {} no se reconoce".format(comando1))

In [7]:
#funcion que lee el OPC y lo guarda en BD
def actualizar():
    global cursor
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=DESKTOP-820175M\SQLEXPRESS;'
                          'Database=capem;'
                          'Trusted_Connection=yes;')

    cursor=conn.cursor()
    
    vars=opc.read(["Applications.Application_1.b1","Applications.Application_1.Program2.int1","Applications.Application_1.Program2.bool1"])
    varsl=[]
    for x in (vars):
        varsl.append(list(x))
    for n, i in enumerate(varsl):
        if i[1] == False:
            varsl[n][1]=0
        elif i[1] == True:
            varsl[n][1]=1

    cursor.execute(f"update capem.dbo.datos set b1 = {varsl[0][1]}, int1 = {varsl[1][1]} where id =1")
    datos=cursor.execute("select * from capem.dbo.datos")
    
    return datos

### Rutina principal

In [8]:
##### Thread 2, Busca comandos
th2=th.Thread(target=comandos,name="th2")
th2.start()

## Loop principal
while comando != close:
    if comando == start:
        datos=actualizar()
        for i in datos:
            print(i)
        cursor.close()
        time.sleep(0.5)   
print("Programa finalizado")


    

Interfaz iniciada
(1, True, 4)
(1, True, 4)
(1, True, 4)
Interfaz detenida
(1, True, 4)
Interfaz iniciada
(1, False, 5)
(1, False, 5)
Interfaz detenida
(1, True, 5)
Cerrando programa


ProgrammingError: Attempt to use a closed cursor.